In [8]:
%pip install sec-edgar-downloader
%pip install investpy
%pip install PyPDF2
%pip install aiohttp aiofiles beautifulsoup4 PyPDF2 langchain
%pip install aiolimiter
%pip install nest_asyncio
%pip install tqdm
%pip install nltk langdetect
%pip install yfinance
%pip install ftfy
%pip install aiofiles
%pip install nltk
!python -m nltk.downloader punkt



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Serban\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd
import yfinance as yf
import os
import time
from datetime import datetime
import random
import logging
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_exponential
from pydantic import BaseModel, ValidationError
import pyarrow.parquet as pq
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
from io import StringIO, BytesIO
import investpy
import json
from typing import List, Dict, Optional
import gzip
import sqlite3
from datetime import datetime, timedelta
import re
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dateutil import parser


In [4]:
#maybe consider FTSE100
#don t forget about cryptos and forex

def get_sp500_tickers():
    """Get S&P 500 tickers with more reliable table parsing"""
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    df = pd.read_html(StringIO(str(table)))[0]
    return df['Symbol'].tolist()

def get_nasdaq_tickers():
    """Get NASDAQ-listed common stocks with proper column handling"""
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
    columns = [
        'Symbol', 'Security Name', 'Market Category', 'Test Issue',
        'Financial Status', 'Round Lot Size', 'ETF', 'NextShares'
    ]
    df = pd.read_csv(url, sep="|", names=columns)
    df = df[:-1]  # Remove summary row
    # Filter out ETFs and test issues
    df = df[(df['Test Issue'] == 'N') & (df['ETF'] == 'N')]
    return df['Symbol'].tolist()

def get_other_tickers():
    """Get NYSE/AMEX stocks with proper column handling"""
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"
    columns = [
        'ACT Symbol', 'Security Name', 'Exchange', 'CQS Symbol',
        'ETF', 'Round Lot Size', 'Test Issue', 'NASDAQ Symbol'
    ]
    df = pd.read_csv(url, sep="|", names=columns)
    df = df[:-1]  # Remove summary row
    # Filter out ETFs and test issues
    df = df[(df['Test Issue'] == 'N') & (df['ETF'] == 'N')]
    return df['ACT Symbol'].tolist()

def get_global_tickers():
    """Get global tickers with error handling"""
    try:
        stocks_df = investpy.stocks.get_stocks()
        # Filter for common stock types (adjust based on available data)
        if 'type' in stocks_df.columns:
            stocks_df = stocks_df[stocks_df['type'] == 'Stock']
        return stocks_df['symbol'].unique().tolist()
    except ImportError:
        print("Global tickers disabled: install investpy (pip install investpy)")
        return []
    except Exception as e:
        print(f"Error fetching global tickers: {str(e)}")
        return []

def get_all_tickers():
    """Combine all sources with error handling"""
    tickers = []

    try:
        tickers += get_sp500_tickers()
        print(f"Found {len(tickers)} S&P 500 tickers")
    except Exception as e:
        print(f"Error fetching S&P 500: {str(e)}")

    try:
        nasdaq = get_nasdaq_tickers()
        tickers += nasdaq
        print(f"Added {len(nasdaq)} NASDAQ tickers")
    except Exception as e:
        print(f"Error fetching NASDAQ: {str(e)}")

    try:
        other = get_other_tickers()
        tickers += other
        print(f"Added {len(other)} NYSE/AMEX tickers")
    except Exception as e:
        print(f"Error fetching NYSE/AMEX: {str(e)}")

    try:
        global_tickers = get_global_tickers()
        tickers += global_tickers
        print(f"Added {len(global_tickers)} global tickers")
    except Exception as e:
        print(f"Error fetching global tickers: {str(e)}")

    # Clean and deduplicate
    clean_tickers = list(set([t.strip().upper() for t in tickers if isinstance(t, str)]))
    return sorted(clean_tickers)



In [ ]:
if __name__ == "__main__":
    all_tickers = get_all_tickers()
    print(f"\nTotal unique tickers: {len(all_tickers)}")



    #!!! separate the global tickers from the important markets so that the code for the filings are done on the important markets



    # Save to file
    # with open("all_tickers.txt", "w") as f:
    #     f.write("\n".join(all_tickers))

    #store_historical_data()



Found 503 S&P 500 tickers
Added 4002 NASDAQ tickers
Added 3158 NYSE/AMEX tickers
Global tickers disabled: install investpy (pip install investpy)
Added 0 global tickers

Total unique tickers: 7159


**The above code works well for retrieving sec filings, whena query is prompted, the model will call the retriver for the sec filings which will be stored in a database either loccal or on the cloud if the sec filings is what is needed, or it will either call the api for the stock price or the api for the analytics- Finnhub**

In [ ]:
# --- Test Execution ---
if __name__ == "__main__":
    filings = fetch_sec_filings("AAPL", filing_type='10-Q' ,max_results=1,save_to_disk=False)
    print(filings)
    print(filings[0]['documents'][0]['text'][2])



In [7]:
import os, re, json, time, asyncio, aiofiles
from io import BytesIO
from typing import List
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader
import aiohttp
from langchain.text_splitter import RecursiveCharacterTextSplitter

# --- CONFIG ---
HEADERS = {
    "User-Agent": "University_Project_Financial_LLM (muie.restante@gmail.com)",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}
SEC_BASE = "https://www.sec.gov"
CIK_LOOKUP = f"{SEC_BASE}/files/company_tickers.json"
SEARCH_URL = f"{SEC_BASE}/cgi-bin/browse-edgar"
DATA_DIR = "sec_filings_second"
DELAY = 1.0
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200
MAX_CONCURRENT_REQUESTS = 5

# --- UTILS ---
async def async_get(session, url, semaphore, retries=3):
    for attempt in range(retries):
        async with semaphore:
            try:
                async with session.get(url, headers=HEADERS, timeout=aiohttp.ClientTimeout(total=30)) as resp:
                    await asyncio.sleep(DELAY)
                    if resp.status == 429:
                        wait_time = 2 ** (attempt + 5)
                        print(f"Rate limited. Retrying in {wait_time}s...")
                        await asyncio.sleep(wait_time)
                        continue
                    content_type = resp.headers.get("Content-Type", "")
                    if "application/json" not in content_type and url.endswith(".json"):
                        print(f"Unexpected content type: {content_type}")
                        return None
                    return await resp.read()
            except Exception as e:
                print(f"Request failed (attempt {attempt+1}): {e}")
                if attempt == retries - 1:
                    return None
                await asyncio.sleep(2 ** (attempt + 1))
    return None

async def download_cik_lookup(session):
    try:
        async with session.get(CIK_LOOKUP, headers=HEADERS) as resp:
            await asyncio.sleep(DELAY)
            if resp.status != 200:
                raise Exception(f"Status {resp.status}")
            content_type = resp.headers.get("Content-Type", "")
            if "application/json" not in content_type:
                raise Exception(f"Unexpected content-type: {content_type}")
            return await resp.json()
    except Exception as e:
        print(f"Failed to fetch CIK lookup: {e}")
        return {}

def get_cik_from_lookup(ticker: str, lookup_data: dict) -> str:
    for entry in lookup_data.values():
        if entry["ticker"].lower() == ticker.lower():
            return str(entry["cik_str"]).zfill(10)
    return None

def split_text(text: str) -> List[str]:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_text(text)

def clean_text(text: str) -> str:
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r"UNITED STATES SECURITIES.*?Washington, D\.C\.\s*\d+", "", text, flags=re.DOTALL)
    text = re.sub(r"Table of Contents.*?(?=Item\s+1)", "", text, flags=re.DOTALL | re.IGNORECASE)
    text = re.sub(r'[0-9]{10,}', '', text)
    text = re.sub(r'[a-fA-F0-9]{32,}', '', text)
    return re.sub(r'\n+', '\n', text.replace("\xa0", " ")).strip()

async def get_filing_urls(session, cik: str, filing_type: str, max_docs=5):
    try:
        params = {
            "action": "getcompany",
            "CIK": cik,
            "type": filing_type,
            "owner": "exclude",
            "count": "100"
        }
        async with session.get(SEARCH_URL, params=params, headers=HEADERS) as resp:
            content = await resp.text()
            soup = BeautifulSoup(content, "html.parser")
            table = soup.find("table", class_="tableFile2")
            filings = []
            if table:
                rows = table.find_all("tr")[1:]
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 4:
                        link = cols[1].find("a")
                        date = cols[3].text.strip()
                        if link:
                            filings.append({
                                "detail_url": f"{SEC_BASE}{link['href']}",
                                "date": date
                            })
                        if len(filings) >= max_docs:
                            break
            return filings
    except Exception as e:
        print(f"Error fetching filings for CIK {cik}: {e}")
        return []

async def extract_text_from_doc(session, doc_url: str, semaphore) -> str:
    raw = await async_get(session, doc_url, semaphore)
    if not raw:
        return ""
    try:
        if doc_url.endswith(".pdf"):
            reader = PdfReader(BytesIO(raw))
            text = "\n".join(p.extract_text() or "" for p in reader.pages)
        else:
            soup = BeautifulSoup(raw, "html.parser")
            text = soup.get_text()
        return clean_text(text)
    except Exception as e:
        print(f"Error extracting text from {doc_url}: {e}")
        return ""

async def process_filing(session, filing, ticker, filing_type, semaphore):
    try:
        raw = await async_get(session, filing["detail_url"], semaphore)
        if not raw:
            return None
        soup = BeautifulSoup(raw, "html.parser")
        table = soup.find("table", class_="tableFile")
        if not table:
            return None
        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")
            if len(cols) >= 4:
                doc_type = cols[3].text.strip().lower()
                href = cols[2].find("a")
                if not href:
                    continue
                href = href["href"]
                filename = href.split("/")[-1].lower()
                if(
                    doc_type == filing_type.lower() and
                    (filename.endswith(".htm") or filename.endswith(".html"))
                    and not any(x in filename for x in ["xbrl", "xml", "ex-99", "exhibit", "form", "index"])
                ):
                    doc_url = f"{SEC_BASE}{href}".replace("/ix?doc=", "") # Corrected line
                    text = await extract_text_from_doc(session, doc_url, semaphore)
                    if not text:
                        continue
                    return {
                        "ticker": ticker,
                        "date": filing["date"],
                        "text": text,
                        "url": doc_url,
                        "filing_type": filing_type
                    }
                        
                    
        return None
    except Exception as e:
        print(f"Error processing filing {filing['detail_url']}: {e}")
        return None

async def process_company(session, ticker, cik, semaphore, filing_type="10-K", max_docs=5):
    filing_dir = os.path.join(DATA_DIR, ticker, filing_type)
    if os.path.exists(filing_dir) and os.listdir(filing_dir):
        print(f"Skipping existing: {ticker} {filing_type}")
        return

    filings = await get_filing_urls(session, cik, filing_type, max_docs)
    if not filings:
        print(f"No filings found for {ticker} {filing_type}")
        return

    for filing in filings:
        path = os.path.join(DATA_DIR, ticker, filing_type, f"{filing['date']}.json")
        if os.path.exists(path):
            continue
        doc = await process_filing(session, filing, ticker, filing_type, semaphore)
        if doc:
            os.makedirs(os.path.dirname(path), exist_ok=True)
            async with aiofiles.open(path, "w") as f:
                await f.write(json.dumps(doc))

async def run_pipeline(tickers):
    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    async with aiohttp.ClientSession() as session:
        cik_lookup = await download_cik_lookup(session)
        if not cik_lookup:
            print("Aborting: Failed to fetch CIK lookup")
            return

        BATCH_SIZE = 50
        for i in range(0, len(tickers), BATCH_SIZE):
            batch = tickers[i:i+BATCH_SIZE]
            print(f"Processing batch {i//BATCH_SIZE + 1}/{(len(tickers)//BATCH_SIZE)+1}")
            tasks = []
            for ticker in batch:
                cik = get_cik_from_lookup(ticker, cik_lookup)
                if not cik:
                    print(f"CIK not found: {ticker}")
                    continue
                tasks.append(process_company(session, ticker, cik, semaphore, "10-K", 5))
                tasks.append(process_company(session, ticker, cik, semaphore, "10-Q", 15))
            await asyncio.gather(*tasks)
            print(f"Finished batch {i//BATCH_SIZE + 1}")




tickers = get_sp500_tickers() + get_nasdaq_tickers() + get_other_tickers()



In [ ]:
print(tickers.__len__())

7637


In [8]:
import nest_asyncio
nest_asyncio.apply()
await run_pipeline(tickers[:10])

Processing batch 1/1
Finished batch 1


In [ ]:
SCRAPING_FILE = "/content/drive/MyDrive/Licenta/earnings_transcripts/motley-fool-data.pkl"
DELAY = 1.0
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

def read_and_clean(scraping_file: str) -> pd.DataFrame:
    """
    Load a pickled Motley Fool earnings-call dump and return a cleaned DataFrame
    suitable for downstream chunking / RAG ingestion.
    """

    # ───────────────────────────────
    # helpers
    # ───────────────────────────────
    def clean_date(raw: str) -> str:
        """Parse a date string and return ISO-8601 format."""
        if not isinstance(raw, str) or not raw.strip():
            return raw
        try:
            # Remove trailing timezone (like "ET")
            no_tz = re.sub(r"\s+[A-Z]{2,4}$", "", raw.strip())
            dt = parser.parse(no_tz, fuzzy=True)
            return dt.isoformat()
        except (ValueError, OverflowError) as exc:
            print(f"[WARN] Could not parse date “{raw}”: {exc}")
            return raw  # fallback to original if cannot parse

    dash_re = re.compile(r"\s*--\s*")

    def normalize_label(line: str) -> str:
        if "--" in line:
            return " - ".join(part.strip() for part in dash_re.split(line))
        return line

    junk_hdr_re = re.compile(
        r"^(duration:|call participants:|questions and answers:|more .* analysis$|all earnings call transcripts$)",
        re.I,
    )

    def clean_transcript(blob: str) -> str:
        """Clean the transcript text."""
        out, prev_blank = [], False
        for raw in blob.splitlines():
            line = raw.rstrip()
            if junk_hdr_re.match(line):
                continue
            if not line:
                if not prev_blank:
                    out.append("")
                prev_blank = True
                continue
            prev_blank = False
            out.append(normalize_label(line))
        return "\n".join(out).strip()

    def extract_participants(text: str) -> list:
        names = set()
        for line in text.splitlines():
            if " - " not in line:
                continue

            name = line.split(" - ")[0].strip()

            if (
                name.startswith("[")                     # tags like [Foreign Speech]
                or "foreign speech" in name.lower()      # safety net
                or name.lower() in {"operator", "analyst"}
                or len(name) < 2 or len(name) > 60
            ):
                continue

            names.add(name)

        return sorted(names)

    # ───────────────────────────────
    # load raw pickle → DataFrame
    # ───────────────────────────────
    with open(scraping_file, "rb") as fh:
        raw_data = pickle.load(fh)

    df = pd.DataFrame(raw_data).dropna(subset=["transcript"])

    # ───────────────────────────────
    # clean fields
    # ───────────────────────────────
    df["date"] = df["date"].apply(clean_date)
    df["exchange"] = df["exchange"].str.split(":").str[0].str.upper().str.strip()
    df["transcript"] = df["transcript"].apply(clean_transcript)
    df["year"] = df["q"].str.extract(r"(\d{4})")
    df["quarter"] = df["q"].str.extract(r"Q([1-4])")
    df["document_type"] = "earnings_call"
    df["language"] = "en"
    df["participants"] = df["transcript"].apply(extract_participants)

    df = df[
        [
            "date",
            "year",
            "quarter",
            "exchange",
            "ticker",
            "q",
            "document_type",
            "language",
            "participants",
            "transcript",
        ]
    ]

    # ───────────────────────────────
    # sanity checks
    # ───────────────────────────────
    if df["transcript"].isnull().any():
        raise ValueError("Null transcripts detected after cleaning.")
    if len(df) != len(raw_data):
        raise ValueError("Row count mismatch after cleaning.")

    print(f"✅ Cleaned {len(df)} records · columns → {list(df.columns)}")
    return df



In [ ]:
cleaned = read_and_clean(SCRAPING_FILE)
for index, row in cleaned.head(2).iterrows():
    for column_name, value in row.items():
        print(f"{column_name}: {value}")
    print("\n")  # Add an extra newline for readability

In [ ]:
#chunk and store the data
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", " "],
)

all_chunks = []

cleaned = read_and_clean(SCRAPING_FILE)
for _, row in cleaned.iterrows():
    for chunk in splitter.split_text(row["transcript"]):
        all_chunks.append(
            {
                "text_chunks": chunk,
                "metadata": {
                    "ticker": row.ticker,
                    "date": row.date,
                    "year": row.year,
                    "quarter": row.quarter,
                    "participants": row.participants,
                }
            }
        )


✅ Cleaned 18755 records · columns → ['date', 'year', 'quarter', 'exchange', 'ticker', 'q', 'document_type', 'language', 'participants', 'transcript']


In [ ]:
import json
import os

SAVE_DIR = "/content/drive/MyDrive/Licenta/earnings_transcripts/companies"

# Group chunks by ticker+year+quarter
transcript_map = {}
for chunk in all_chunks:
    meta = chunk['metadata']
    key = f"{meta['ticker']}_{meta['year']}_Q{meta['quarter']}"

    if key not in transcript_map:
        transcript_map[key] = {
            "metadata": {
                "ticker": meta['ticker'],
                "date": meta['date'],
                "year": meta['year'],
                "quarter": meta['quarter'],
                "participants": meta['participants']
            },
            "chunks": []
        }

    # Just add chunk text with ID
    transcript_map[key]['chunks'].append({
        "id": len(transcript_map[key]['chunks']) + 1,
        "text": chunk['text_chunks']
    })

# Save each group as a separate file
for key, transcript in transcript_map.items():
    ticker = transcript['metadata']['ticker']
    year = transcript['metadata']['year']
    quarter = transcript['metadata']['quarter']

    # Create company directory
    company_dir = os.path.join(SAVE_DIR, ticker)
    os.makedirs(company_dir, exist_ok=True)

    # Save as JSON
    filename = f"{ticker}_{year}_Q{quarter}.json"
    with open(os.path.join(company_dir, filename), 'w') as f:
        json.dump(transcript, f, indent=2)

In [ ]:
for i, chunk in enumerate(all_chunks):
  print(f"Chunk {i + 1}:")
  print("-" * 20)
  print(f"Content: {chunk['text_chunks']}")
  print("-" * 20)
  print("Metadata:")
  for key, value in chunk['metadata'].items():  # Iterate through metadata
    print(f"  {key}: {value}")  # Print key-value pairs
  print("\n")

In [ ]:
#This worked !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
import re, json, ftfy, os
from pathlib import Path
from nltk.tokenize import sent_tokenize   # run `python -m nltk.downloader punkt` once
from dateutil import parser
import logging

# ---------- 1. header/footer patterns ----------
HEADER_FOOTER_PATTERNS = [
    re.compile(r'^\s*TABLE OF CONTENTS\s*$',                 re.I),
    re.compile(r'^\s*INDEX TO FINANCIAL STATEMENTS\s*$',     re.I),
    re.compile(r'^\s*F-\d+\s*$|^\s*\d+\s*$',                 re.I),          # “F-3” or bare page numbers
    re.compile(r'^\s*(10-K|10-Q)\b.*$',                      re.I),          # “10-K 1 …”
    re.compile(r'^\s*Item\s+\d+[A-Z]?\.?.*\s+\d+\s*$',       re.I),          # “Item 1. Business 1”
    re.compile(r'^\s*THE ORIGINAL BARK COMPANY.*$',          re.I),          # company-specific header
]

def strip_headers_footers(text: str) -> str:
    lines = []
    for ln in text.splitlines():
        if any(pat.match(ln) for pat in HEADER_FOOTER_PATTERNS):
            continue
        lines.append(ln)
    return "\n".join(lines)

# ---------- 2. sentence-level cleaner ----------
def clean_text(text: str) -> str:
    text = ftfy.fix_text(text.replace('�', "'").replace('’', "'"))

    # split camelCase but keep “U.S.”
    text = re.sub(r'(?<=[a-z])(?=[A-Z][a-z])', ' ', text)

    # collapse whitespace
    text = re.sub(r'[\n\r\t]+', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text).strip()
    #added the item 1 strip!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    
    # sentence split (NLTK if available, regex otherwise)
    sents = sent_tokenize(text) if sent_tokenize else re.split(r'(?<=[.!?])\s+(?=[A-Z(])', text)

    dedup, seen = [], set()
    for s in sents:
        norm = re.sub(r'\s+', ' ', s).strip().lower()
        if norm and norm not in seen:
            dedup.append(s.strip())
            seen.add(norm)

    return ' '.join(dedup)

# ---------- 3. main one-file cleaner ----------
def process_file(path: Path, dest_root: Path):
    data = json.loads(path.read_text(encoding="utf-8"))
    chunks = data.get("text_chunks", [])
    if data.get("filing_type") == "10-K":
        chunks = chunks[1:]                     # drop SEC header chunk

    # -------- cross-chunk de-duplication -----------------------------------
    joined, seen = [], set()
    for ch in chunks:
        if not ch or "us-gaap" in ch.lower():
            continue
        norm = re.sub(r'\s+', ' ', ch).strip().lower()
        if norm in seen:
            continue
        seen.add(norm)
        joined.append(ch.strip())

    full_text = ' '.join(joined)

    # -------- hard-cut everything before “ITEM 1.” -------------------------
    # m = re.search(r'(?i)\bITEM\s+1\.\s', full_text)
    m = re.search(
        r'''(?is)                 # i = ignore-case, s = dot matches newline
            Item                  # literally “Item” (no word-boundary)
            \s*1                  # spaces (optional) then the digit 1
            \s*[\.\u2022]         # a dot or the bullet ● some filings use
            (?!\d)                # NOT followed by another digit (skips Item 10.)
        ''',
        full_text,
        re.VERBOSE
        )
    if m:
        full_text = full_text[m.start():]
    else:
        print(f"Warning: 'Item 1.' not found in {path.name}")

    full_text = strip_headers_footers(full_text)
    full_text = clean_text(full_text)

    # -------- drop tiny leftovers -----------------------------------------
    MIN_WORDS = 50
    if len(full_text.split()) < MIN_WORDS:
        logging.info("Dropped %s (only %d words)", path, len(full_text.split()))
        return                         # nothing written

    # =======================  METADATA BLOCK  ==============================
    date_str = data.get("date")                       # e.g. "2022-12-21"
    try:
        year_of_filing = parser.parse(date_str).year if date_str else None
    except Exception as e:
        print(f"Could not parse year from {date_str}: {e}")
        year_of_filing = None

    source_url   = data.get("url")                    # e.g. ".../data/1090872/..."
    cik_match    = re.search(r"/data/(\d+)/", source_url or "")
    extracted_cik = cik_match.group(1).zfill(10) if cik_match else None

    out = {
        "cleaned_text": full_text,
        "ticker":      data.get("ticker"),
        "date":        date_str,
        "year":        year_of_filing,
        "cik":         extracted_cik,
        "filing_type": data.get("filing_type"),
    }
    # ======================================================================

    # mirror raw-path subtree under sec_filings_clean/
    out_path = dest_root / path.relative_to(RAW_ROOT)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(json.dumps(out, ensure_ascii=False, indent=2),
                        encoding="utf-8")

# ---------- 4. batch driver ----------
# -----------------------------------------
# 5.  ultra-simple batch driver (no Pool) 
# # -----------------------------------------
if __name__ == "__main__":
    import glob, sys
    from tqdm import tqdm

    RAW_ROOT   = Path("sec_filings")
    CLEAN_ROOT = Path("sec_filings_clean")

    files = glob.glob(str(RAW_ROOT / "**/*.json"), recursive=True)
    if not files:
        sys.exit("No raw filings found under sec_filings/")

    errors = 0
    for fp in tqdm(files, desc="Cleaning filings"):
        try:
            process_file(Path(fp), CLEAN_ROOT)
            print(f"Cleaned: {fp}")
        except Exception as e:
            errors += 1
            with open("clean_errors.log", "a", encoding="utf-8") as log:
                log.write(f"{fp}\t{e}\n")

    print(f"\nDone.  {len(files)-errors} cleaned, {errors} errors (see clean_errors.log)")





In [3]:
import gzip, csv, itertools, aiohttp, asyncio, re, json, os, textwrap

FORM_TYPES  = {"10-K", "10-Q"}

# Define DATA_ROOT, used in fetch_and_store
DATA_ROOT = "test" # Or your desired path

# --- CONFIG for CIK lookup ---
CIK_LOOKUP_URL = "https://www.sec.gov/files/company_tickers.json"
LOOKUP_HEADERS = {
    "User-Agent": "University_Project_Financial_LLM (muie.restante@gmail.com)",
    "Accept-Encoding": "gzip, deflate",
    "Host": "www.sec.gov"
}
DELAY_LOOKUP = 0.1 # Adjusted delay for a single request

async def download_ciks(session):
    """Downloads the CIK lookup file and extracts all CIK numbers."""
    try:
        print("Fetching CIK lookup data...")
        async with session.get(CIK_LOOKUP_URL, headers=LOOKUP_HEADERS) as resp:
            await asyncio.sleep(DELAY_LOOKUP) # Be respectful to SEC servers
            if resp.status != 200:
                print(f"Failed to fetch CIK lookup: Status {resp.status}")
                return []
            content_type = resp.headers.get("Content-Type", "")
            if "application/json" not in content_type:
                print(f"Unexpected content-type for CIK lookup: {content_type}")
                return []
            data = await resp.json()
            ciks = [str(entry["cik_str"]).zfill(10) for entry in data.values()]
            print(f"Successfully fetched {len(ciks)} CIKs.")
            return ciks
    except Exception as e:
        print(f"Error during CIK lookup download: {e}")
        return []

async def initialize_ciks():
    """Initializes the CIK list."""
    async with aiohttp.ClientSession() as session:
        return await download_ciks(session)

my_cik_list = [] # Will be populated by initialize_ciks
CIK_SET     = set(my_cik_list)              # 4–10 k entities
QUARTER_URL = "https://www.sec.gov/Archives/edgar/full-index/2025/Q2/master.gz" # Current date is May 16, 2025. Adjusted to Q2 2025.
UA = "MyResearchBot/1.0 (Muie; muie.restante@gmail.com) requests for academic research - contact for questions"

async def master_rows():
    if not CIK_SET:
        print("CIK_SET is empty. No data to fetch.")
        return
    headers = {
    "User-Agent": UA,
    "Accept": "*/*",
    "Referer": "https://www.sec.gov/",
    "Host": "www.sec.gov"
        }
    async with aiohttp.ClientSession(headers=headers) as s:
        async with s.get(QUARTER_URL, timeout=30) as r:
            if r.status != 200:
                print(f"Failed to fetch master file: Status {r.status}")
                return
            try:
                content = await r.read()
                decompressed_content = gzip.decompress(content).decode()
                rows = decompressed_content.splitlines()
                 # Find the header line index dynamically
                header_line_index = -1
                for i, line_content in enumerate(rows):
                    if line_content.startswith("CIK|Company Name|Form Type|Date Filed|File Name"):
                        header_line_index = i
                        break
                
                if header_line_index == -1:
                    print("Master file header not found. Cannot parse.")
                    return

                # Start processing from the line after the header
                relevant_rows = rows[header_line_index + 1:]

            except gzip.BadGzipFile:
                print(f"Error decompressing master file from {QUARTER_URL}. It might be corrupted or not gzipped.")
                # Attempt to read as plain text if decompression fails and it's small
                if len(content) < 1024 * 1024: # 1MB threshold
                    try:
                        plain_text_content = content.decode()
                        rows = plain_text_content.splitlines()
                        header_line_index = -1
                        for i, line_content in enumerate(rows):
                            if line_content.startswith("CIK|Company Name|Form Type|Date Filed|File Name"):
                                header_line_index = i
                                break
                        if header_line_index == -1:
                            print("Master file header not found in plain text attempt.")
                            return
                        relevant_rows = rows[header_line_index + 1:]
                        print("Successfully read master file as plain text after gzip error.")
                    except Exception as plain_text_err:
                        print(f"Could not read master file as plain text either: {plain_text_err}")
                        return
                else:
                    return # Don't process if large and ungzipped

    for row_data in csv.reader(relevant_rows, delimiter="|"):
        if len(row_data) < 5: # Ensure row has enough columns
            continue
        cik, _, form, date, path = row_data
        if form in FORM_TYPES and cik in CIK_SET:
            yield cik.zfill(10), form, date, f"https://www.sec.gov/Archives/{path}"

async def fetch_and_store(sema, row_info):
    cik, form, date, url = row_info
    # Ensure DATA_ROOT is defined and accessible
    if not DATA_ROOT:
        print("DATA_ROOT is not defined. Cannot save file.")
        return

    dir_path = os.path.join(DATA_ROOT, cik, form)
    fname = os.path.join(dir_path, f"{date}.txt")
    
    if os.path.exists(fname):                                 # incremental run
        # print(f"File {fname} already exists. Skipping.")
        return
    
    print(f"Fetching: {url}")
    try:
        async with sema, aiohttp.ClientSession(headers={"User-Agent": UA}) as s:
            await asyncio.sleep(DELAY_LOOKUP) # Respect SEC rate limits
            async with s.get(url, timeout=60) as r: # Increased timeout for potentially large files
                if r.status == 200:
                    txt = (await r.read()).decode("utf-8", "ignore")
                    os.makedirs(dir_path, exist_ok=True)
                    with open(fname, "w", encoding="utf-8") as f:
                        f.write(txt)
                    print(f"Stored: {fname}")
                elif r.status == 404:
                    print(f"File not found (404): {url}")
                elif r.status == 403:
                    print(f"Access forbidden (403) for {url}. Check User-Agent or IP restrictions.")
                else:
                    print(f"Failed to fetch {url}: Status {r.status}")
    except asyncio.TimeoutError:
        print(f"Timeout fetching {url}")
    except Exception as e:
        print(f"Error fetching or storing {url}: {e}")


async def main():
    global my_cik_list, CIK_SET
    # Initialize CIKs first
    fetched_ciks = await initialize_ciks()
    if not fetched_ciks:
        print("No CIKs were fetched. Exiting.")
        return
    
    my_cik_list = fetched_ciks
    CIK_SET = set(my_cik_list)
    print(f"Working with {len(CIK_SET)} unique CIKs.")

    # Limit the number of CIKs for testing if needed
    CIK_SET = set(list(CIK_SET)[:1]) # Example: use only the first 10 CIKs
    print(f"DEBUG: Using a subset of {len(CIK_SET)} CIKs for processing.")


    if not CIK_SET:
        print("CIK list is empty after initialization. Cannot proceed.")
        return

    sema = asyncio.Semaphore(10) # Adjusted semaphore based on SEC's 10 req/s limit
    
    tasks = []
    async for row_details in master_rows():
        if row_details: # Ensure row_details is not None
            tasks.append(fetch_and_store(sema, row_details))
    
    if tasks:
        print(f"Gathered {len(tasks)} tasks to process.")
        await asyncio.gather(*tasks)
        print("All tasks completed.")
    else:
        print("No tasks to process based on the master file and CIK list.")

if __name__ == '__main__': # Required for Colab/Jupyter if running asyncio directly
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(main())

Fetching CIK lookup data...
Successfully fetched 10039 CIKs.
Working with 7880 unique CIKs.
DEBUG: Using a subset of 1 CIKs for processing.
Failed to fetch master file: Status 403
No tasks to process based on the master file and CIK list.


In [3]:
import os
# Set the cache directory to a folder on your D: drive
# Make sure this path exists or can be created.
os.environ['HF_DATASETS_CACHE'] = 'D:\\huggingface_cache\\datasets' # Example path on D:

from datasets import load_dataset, DatasetDict
# Load the dataset for the 'train' split
# The original command includes DatasetDict in its import, so we keep it.
edgar_dataset_train = load_dataset('eloukas/edgar-corpus', split='train', trust_remote_code=True)
# Save it to a CSV file
edgar_dataset_train.to_csv('edgar_corpus_10k.csv')

# You can optionally print the cache directory to confirm
from datasets.config import HF_DATASETS_CACHE
print(f"Datasets cache directory is: {HF_DATASETS_CACHE}")

Creating CSV from Arrow format: 100%|██████████| 177/177 [09:01<00:00,  3.06s/ba]

Datasets cache directory is: C:\Users\Serban\.cache\huggingface\datasets


In [ ]:
import pandas as pd

chunksize = 10  # rows per chunk, tune based on your system
for chunk in pd.read_csv("edgar_corpus_10k.csv", chunksize=chunksize):
    # Process each chunk
    print(chunk.head())  # Example: preview first few rows of each chunk


In [3]:
import pandas as pd

# Step 1: Find column names without loading the whole file
df_sample = pd.read_csv("edgar_corpus_10k.csv", nrows=5)
print(df_sample.columns)  # See what you're working with


Index(['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B',
       'section_2', 'section_3', 'section_4', 'section_5', 'section_6',
       'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A',
       'section_9B', 'section_10', 'section_11', 'section_12', 'section_13',
       'section_14', 'section_15'],
      dtype='object')


In [12]:
import pandas as pd

# Define output file and sections of interest
output_file = "sample_filings.txt"
usecols = ['filename', 'cik', 'year', 'section_1', 'section_1A', 'section_1B',
       'section_2', 'section_3', 'section_4', 'section_5', 'section_6',
       'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A',
       'section_9B', 'section_10', 'section_11', 'section_12', 'section_13',
       'section_14', 'section_15']
chunksize = 1
num_samples = 5  # how many filings you want to store

with open(output_file, "w", encoding="utf-8") as out:
    for i, chunk in enumerate(pd.read_csv("edgar_corpus_10k.csv", usecols=usecols, chunksize=chunksize)):
        row = chunk.iloc[0]
        out.write(f"\n\n=== Filing {i + 1} ===\n")
        out.write(f"Filename: {row['filename']}\n")
        out.write(f"CIK: {row['cik']}\n")
        out.write(f"Year: {row['year']}\n")

        for sec in ['section_1', 'section_1A', 'section_1B',
       'section_2', 'section_3', 'section_4', 'section_5', 'section_6',
       'section_7', 'section_7A', 'section_8', 'section_9', 'section_9A',
       'section_9B', 'section_10', 'section_11', 'section_12', 'section_13',
       'section_14', 'section_15']:
            out.write(f"\n--- {sec} ---\n")
            section_text = str(row[sec])
            # You can write full text or limit it:
            out.write(section_text)  # adjust char limit as needed
        out.write("\n" + "=" * 20 + "\n")

        if i + 1 == num_samples:
            break

print(f"Saved {num_samples} filings to {output_file}")


Saved 5 filings to sample_filings.txt
